In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import dotenv 
dotenv.load_dotenv()
import sys
import os
sys.path.append(os.path.abspath('../'))
import re
from src import soporte_scrap as sc

In [6]:
driver = webdriver.Chrome()
url = 'https://www.ahorramas.com/'
driver.get(url)
driver.maximize_window()         
sleep(2)
driver.find_element(By.XPATH, '/html/body/div[7]/div[2]/div/div/div[2]/div/div/button[2]').click()        #boton cookies 
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#frescos > span').click()   #productos frescos
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#product-search-results > div:nth-child(2) > div.col-sm-12.col-lg-9 > div.row.product-grid > div.col-12.grid-footer > div > div > button').click()          #mas resultados
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#product-search-results > div:nth-child(2) > div.col-sm-12.col-lg-9 > div.row.product-grid > div.col-12.grid-footer > div > div > button').click()          #mas resultados
html_page = driver.page_source
sopa = BeautifulSoup(html_page)

In [16]:
sopa_productos = sopa.findAll('div', class_ = 'tile-body')


In [17]:
lista_productos=sc.productos_frescos(sopa_productos)

In [18]:
lista_productos
df = pd.DataFrame(lista_productos, columns=["Nombre y peso", "Precio", "Descuentos",'rango de precios bajada'])


In [19]:
df.head(2)

,Nombre y peso,Precio,Descuentos,rango de precios bajada
0,LOTE 2 kg de pechugas de pollo,"\n \n 6,39€\n\n\n",-20%,"\n\n\n (6,39€/Kg)\n ..."
1,Salmón por medios o enteros peso aproximado pi...,"\n \n 10,99€\n\n\n",-8%,"\n\n\n (10,99€/Kg)\n ..."


In [20]:
df['Precio'] = df['Precio'].str.replace('\n', '').str.replace('€', '').str.replace(',', '.').astype(float)  #precio a float
df['rango de precios bajada'] = df['rango de precios bajada'].str.replace('\n', '')


In [21]:
df['rango de precios bajada'] = df['rango de precios bajada'].str.replace('\n', '').str.replace('/','').str.replace('kg','')

In [22]:
df[['precio_actual', 'precio_sin_descuento']] = df['rango de precios bajada'].str.extract(r'\(([\d,]+)€Kg\)\s*\|\s*\(([\d,]+)€Kg\)')
df['precio_sin_descuento'] = df['precio_sin_descuento'].fillna(0)
df['precio_actual'] = df['precio_actual'].fillna(0)
df = df.drop(columns=['rango de precios bajada'])

In [23]:
df['precio_sin_descuento'] = df['precio_sin_descuento'].str.replace(',','.').astype(float)
df['precio_actual'] = df['precio_actual'].str.replace(',','.').astype(float)

In [24]:
df['Descuentos'] = df['Descuentos'].str.replace('%','').str.replace(',','.').astype(float)

In [25]:
df

,Nombre y peso,Precio,Descuentos,precio_actual,precio_sin_descuento
0,LOTE 2 kg de pechugas de pollo,6.39,-20.0,6.39,7.99
1,Salmón por medios o enteros peso aproximado pi...,10.99,-8.0,10.99,11.99
2,Lubina de ración pieza 400g aprox 2 uds,6.99,-30.0,NaN,NaN
3,LOTE 2kg de contramuslos de pollo,4.79,-20.0,4.79,5.99
4,LOTE 2kg de alas de pollo,3.99,-20.0,3.99,4.99
5,Cordero lechal por medios,16.99,-15.0,16.99,19.99
6,Filetes de añojo 1ªB,12.99,-16.0,12.99,15.49
7,Ragout de añojo,9.49,-27.0,9.49,12.99
8,LOTE: 250gr jamon GR + 250gr chorizo + 250gr q...,16.00,-17.0,16.00,19.33
9,LOTE: 250gr jamón asado + 250gr pavo + 250gr q...,11.33,-19.0,11.33,13.99


In [26]:
df.to_csv('../datos/datos_scrap_ahorra.csv', index=False)